In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
with open("received_messages/125525_binary.txt", "r") as f:
    bitstring = f.read()
bitstring_chars = [*bitstring]
bits = [int(x) for x in bitstring_chars]
df = pd.DataFrame(data={"Bit": bits})

In [3]:
# Make plot of the bit sequence
fig = px.line(df, y="Bit")
fig.update_layout(
    template="plotly_dark",
    # xaxis_range=[0,100],
)
fig.show()

In [4]:
# Do correlation of the start sequence and the received sequence
start_seq = [1,0,0,1,0,0,1,1,0,1,0,0,1,1,1,1,0,1,1,1,0,0,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,0,1,1,0,0,0,1,0,1,0,0,1,0,1,1,1,1,1,0,1,0,1,0,1,0,0,0,0,1,0,1,1,0,1,1,1,1,0,0,1,1,1,0,0,1,0,1,0,1,1,0,0,1,1,0,0,0,0,0,1,1,0,1,1,0,1,0,1,1,1,0,1,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0]
corr = np.correlate(bits, start_seq)
df2 = pd.DataFrame(data={"xcorr": corr})

fig = px.line(df2, y="xcorr")
fig.update_layout(
    template="plotly_dark",
    # xaxis_range=[0,100],
)
fig.show()
maximum = df2.idxmax(axis=0).values[0]
print(f"The preamble starts at bit {maximum}")

The preamble starts at bit 54


In [5]:
# Extract the remaining portion of the message after the start sequence
message = bits[maximum + len(start_seq):] # need to offset preamble length
message_bytes = list()
c = 0
curr_byte = ""
for bit in message:
    curr_byte += str(bit)
    c += 1
    if c == 8:
        message_bytes.append(curr_byte)
        c = 0
        curr_byte = ""
message_literals = [int(bitstring, 2) for bitstring in message_bytes]
message_chars = [chr(i) for i in message_literals]

message = "".join(message_chars)
message

'@The rain in spain stays mainly in the plains.\nI dreamed a dream of a time gone by.\nWhy should a woman who is healthy and strong blubber like a baby if her mans goes away?\nThe phantom of the opera '

In [6]:
# Hacky solution (didn't get pushed to git for some reason?)
message_firsthalf = bits[maximum + len(start_seq):]
message_secondhalf = bits[:maximum-1]
message_bytes = list()
c = 0
curr_byte = ""
for bit in list(message_firsthalf + message_secondhalf):
    curr_byte += str(bit)
    c += 1
    if c == 8:
        message_bytes.append(curr_byte)
        c = 0
        curr_byte = ""
message_literals = [int(bitstring, 2) for bitstring in message_bytes]
message_chars = [chr(i) for i in message_literals]
message = "".join(message_chars)
message

'@The rain in spain stays mainly in the plains.\nI dreamed a dream of a time gone by.\nWhy should a woman who is healthy and strong blubber like a baby if her mans goes away?\nThe phantom of the opera \x1bw?ÚN\x15'